### Imports

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import fiona
import shapely.geometry
import common
from shapely.geometry import Point, Polygon
from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

### Inputs that will change

In [2]:
number_of_stations = 5
date = '09/24/2017'

### Make df of all subway stations

In [3]:
df_columns = ['STATION_ID', 'COMPLEX_ID', 'GTFS_STOP_ID', 'DIVISION', 'LINE', 'STOP_NAME', 'BOROUGH', 'DAYTIME_ROUTES', 'STRUCTURE', 'LATITUDE', 'LONGITUDE']
df_train_stops = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')
df_train_stops.columns=df_columns

In [4]:
df_train_stops[df_train_stops['STOP_NAME'].str.contains('Grand Cen')]

,STATION_ID,COMPLEX_ID,GTFS_STOP_ID,DIVISION,LINE,STOP_NAME,BOROUGH,DAYTIME_ROUTES,STRUCTURE,LATITUDE,LONGITUDE
400,402,610,631,IRT,Lexington - Shuttle,Grand Central - 42 St,M,4 5 6,Subway,40.751776,-73.976848
463,465,610,723,IRT,Flushing,Grand Central - 42 St,M,7,Subway,40.751431,-73.976041
467,469,610,901,IRT,Lexington - Shuttle,Grand Central - 42 St,M,S,Subway,40.752769,-73.979189


### Load cleaned turnstile data - CHECK PATH

In [5]:
path = '/Users/murdock/Documents/Metis/MTABenson_metis/pklfiles/'
with open(path + 'another100_final.pkl', 'rb') as pickle_file:
    df = pickle.load(pickle_file)

In [6]:
df.head()

,DATE,STATION,LINENAME,SCP,TIME,ENTRIES,EXITS,ENTRY_DIFF,EXIT_DIFF,HR_DELTA,WEEKDAY,MAX,EN_HRLY_RATE,EX_HRLY_RATE,DATE_OBJ
1,04/30/2016,59 ST,NQR456,02-00-00,4,5639991,1909993,50.0,10.0,4.0,5,2880.0,12.50,2.50,2016-04-30
2,04/30/2016,59 ST,NQR456,02-00-00,8,5640014,1910024,23.0,31.0,4.0,5,2880.0,5.75,7.75,2016-04-30
3,04/30/2016,59 ST,NQR456,02-00-00,12,5640158,1910134,144.0,110.0,4.0,5,2880.0,36.00,27.50,2016-04-30
4,04/30/2016,59 ST,NQR456,02-00-00,16,5640454,1910197,296.0,63.0,4.0,5,2880.0,74.00,15.75,2016-04-30
5,04/30/2016,59 ST,NQR456,02-00-00,20,5640802,1910254,348.0,57.0,4.0,5,2880.0,87.00,14.25,2016-04-30


### Function to select top stations on a given weekday

In [7]:

def select_stations(date, number_of_stations, df):
    """
    Returns a dataframe with the top stations based on total entries and exits per station on a given day.
    
    Inputs:
    date: the date people will be handing out flyers, a string in the form mm/dd/yyyy
    number_of_stations: the number of stations where people will be handing out flyers
    df: a dataframe of cleaned turnstile data
    
    Returns:
    A dataframe with the weekday, linename, station, and total number of entries and exits
    """
    desired_date = datetime.strptime(date, '%m/%d/%Y').weekday()
    df.dropna(how = 'any', inplace=True)
    df['TOTAL'] = df['ENTRY_DIFF'] + df['EXIT_DIFF']
    extract_data_df = df[['DATE_OBJ', 'STATION', 'SCP', 'TOTAL', 'LINENAME', 'WEEKDAY']]
    sorted_df = extract_data_df.sort_values(['WEEKDAY', 'TOTAL'], ascending=[True,False])
    grouped_df = (sorted_df
                  .groupby(['WEEKDAY', 'LINENAME', 'STATION'])
                  .sum().
                  reset_index().
                  sort_values(['TOTAL'], ascending=False)
                 )
    station_df = (grouped_df[grouped_df['WEEKDAY']==desired_date][['WEEKDAY', 'LINENAME', 'STATION', 'TOTAL']]
                  .iloc[:number_of_stations + 50]
                 )
    return station_df
    
stations = select_stations(date, 5, df)

In [8]:
stations.head()

,WEEKDAY,LINENAME,STATION,TOTAL
3326,6,BDFMNQRW,34 ST-HERALD SQ,3.268863e+09
3459,6,L,1 AV,1.788441e+09
3139,6,25,BEVERLY RD,1.336935e+09
3333,6,BQ,AVENUE M,2.684398e+08
3186,6,4567S,GRD CNTRL-42 ST,8.652050e+07


### Function to obtain latitude and longitude for each top station

In [9]:
train_stops_for_mapping = df_train_stops['STOP_NAME']
train_lines_for_mapping = df_train_stops['DAYTIME_ROUTES']

In [10]:
def find_lat_and_long(station_df):
    """
    Returns a list of stations, stop names, and the corresponding lat and long
    
    Inputs:
    station_df: a dataframe of weekday, linename, station, and total number of entries and exits.
    
    Outputs:
    a nested list of plotting data of the format station, stop_name, lat, and long
    """
    plotting_data = []
    for i, station in enumerate(station_df['STATION']):
        stop = process.extractOne(station, train_stops_for_mapping, scorer=fuzz.token_set_ratio)
        routes = df_train_stops[df_train_stops['STOP_NAME'] == stop[0]]['DAYTIME_ROUTES']
        line = [li for li in station_df['LINENAME']]
        line2 = process.extractOne(line[i], routes, scorer=fuzz.token_set_ratio)
        lats = df_train_stops[(df_train_stops['STOP_NAME']==stop[0])&(df_train_stops['DAYTIME_ROUTES']==line2[0])]['LATITUDE']
        longs = df_train_stops[(df_train_stops['STOP_NAME']==stop[0])&(df_train_stops['DAYTIME_ROUTES']==line2[0])]['LONGITUDE']
        plotting_data.append([stop[0], line2[0], [lat for lat in lats][0], [long for  long in longs][0]])
    return plotting_data


station_coord = find_lat_and_long(stations)
station_coord_df = pd.DataFrame(station_coord, columns=['STATION', 'STOP_NAME', 'LAT', 'LONG'])

In [11]:
station_coord_df.head()

,STATION,STOP_NAME,LAT,LONG
0,34 St - Herald Sq,N Q R W,40.749567,-73.987950
1,1 Av,L,40.730953,-73.981628
2,Beverly Rd,2 5,40.645098,-73.948959
3,Avenue M,B Q,40.617618,-73.959399
4,Grand Central - 42 St,4 5 6,40.751776,-73.976848


### Taking top matches and determining block and median income. CHECK FILENAME PATH

In [12]:
filename = '/Users/murdock/Documents/metis/MTABenson_metis/thematic_map_shape/nyshapefile.shp'
blocks = []
with fiona.open(filename) as data:
    for block_data in data:
        geometry = shapely.geometry.shape(block_data['geometry'])
        block = block_data['properties']['GEO_ID']
        blocks.append([block, block_data['geometry']['coordinates']])

In [13]:
latitudes = [lat for lat in station_coord_df['LAT']]
longitudes = [long for long in station_coord_df['LONG']]
stations = [location for location in station_coord_df['STATION']]
stops = [stop for stop in station_coord_df['STOP_NAME']]
coordinates = list(zip(stations, stops, longitudes, latitudes))

In [14]:
geo_id = []
for block in blocks:
    if len(block[1][0]) >= 3:
        poly = Polygon(block[1][0])
        for coord in coordinates:
            point = Point(coord[2:])
            if poly.contains(point):
                #print(block[0])
                geo_id.append([coord, block[0]])
    
print(len(geo_id), len(coordinates))
print(geo_id[0])

50 55
[('59 St', 'N R', -74.017881000000003, 40.641362000000001), '1500000US360470074002']


### Make a df of stations and which block from census data they are located within

In [15]:
columns=['STATION', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'GEO_ID']
geo_data = []
for stations in geo_id:
    station = stations[0][0]
    stop = stations[0][1]
    lat = stations[0][3]
    lon = stations[0][2]
    geo = stations[1]
    geo_data.append([station, stop, lat, lon, geo])
income_df = pd.DataFrame(geo_data, columns=columns)      


In [16]:
income_df.head()

,STATION,STOP_NAME,LATITUDE,LONGITUDE,GEO_ID
0,59 St,N R,40.641362,-74.017881,1500000US360470074002
1,Times Sq - 42 St,1 2 3,40.755290,-73.987495,1500000US360610113001
2,Times Sq - 42 St,N Q R W,40.754672,-73.986754,1500000US360610113001
3,42 St - Bryant Pk,B D F M,40.754222,-73.984569,1500000US360610113001
4,Coney Island - Stillwell Av,D F N Q,40.577422,-73.981233,1500000US360470350002


### Load census data and merge the df with the previous one to obtain median income near each station. CHECK TABLE PATH

In [17]:
income_table = pd.read_csv('/Users/murdock/Documents/metis/MTABenson_metis/ACS_15_5YR_B19013_with_ann.csv')

In [18]:
income_table = income_table.drop(income_table.index[0])
income_table.columns=['GEO_ID', 'GEO_ID2', 'GEO_DISPLAY_LABEL', 'MEDIAN_INCOME', 'MARGIN_OF_ERROR']
income_table.head()

,GEO_ID,GEO_ID2,GEO_DISPLAY_LABEL,MEDIAN_INCOME,MARGIN_OF_ERROR
1,1500000US360050001000,360050001000,"Block Group 0, Census Tract 1, Bronx County, N...",-,**
2,1500000US360050001001,360050001001,"Block Group 1, Census Tract 1, Bronx County, N...",-,**
3,1500000US360050002000,360050002000,"Block Group 0, Census Tract 2, Bronx County, N...",-,**
4,1500000US360050002001,360050002001,"Block Group 1, Census Tract 2, Bronx County, N...",61344,14654
5,1500000US360050002002,360050002002,"Block Group 2, Census Tract 2, Bronx County, N...",71768,27738


In [19]:
final_station_selection_df = pd.merge(income_df, income_table, on='GEO_ID', how='left')

In [20]:
final_station_selection_df.head()

,STATION,STOP_NAME,LATITUDE,LONGITUDE,GEO_ID,GEO_ID2,GEO_DISPLAY_LABEL,MEDIAN_INCOME,MARGIN_OF_ERROR
0,59 St,N R,40.641362,-74.017881,1500000US360470074002,360470074002,"Block Group 2, Census Tract 74, Kings County, ...",58804,12572
1,Times Sq - 42 St,1 2 3,40.755290,-73.987495,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
2,Times Sq - 42 St,N Q R W,40.754672,-73.986754,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
3,42 St - Bryant Pk,B D F M,40.754222,-73.984569,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
4,Coney Island - Stillwell Av,D F N Q,40.577422,-73.981233,1500000US360470350002,360470350002,"Block Group 2, Census Tract 350, Kings County,...",28359,20779


In [21]:
final_station_selection_df.to_pickle('income_mta_sunday.pkl')

### Selects the top stations based on median income

In [22]:
def top_stations(number_of_stations, df):
    """
    Selects stations that have the highest median income
    
    Inputs:
    number_of_stations: the number of stations where people will be handing out flyers
    df: a dataframe consisting of stations, lat and long, geo_id, and median incomes for the area
    
    Returns:
    a dataframe of the top stations based on income
    """
    selection_df = final_station_selection_df[['STATION', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'MEDIAN_INCOME']]
    candidates = selection_df.sort_values('MEDIAN_INCOME', ascending=False)[:number_of_stations]
    return candidates

stations = top_stations(number_of_stations, final_station_selection_df)

In [23]:
stations.head()

,STATION,STOP_NAME,LATITUDE,LONGITUDE,MEDIAN_INCOME
17,Delancey St,F,40.718611,-73.988114,93643
38,103 St,B C,40.796092,-73.961454,92121
21,Bedford Av,L,40.717304,-73.956872,90370
15,2 Av,F,40.723402,-73.989938,89398
42,Grand St,B D,40.718267,-73.993753,86957


In [24]:
stations.to_pickle('final_stations.pkl')

### Determining top stations for a Tuesday

In [27]:
path = '/Users/murdock/Documents/Metis/MTABenson_metis/pklfiles/'
tues_data = pd.read_pickle(path + 'tuestop.pkl')

In [28]:
tues_data.head()

,WEEKDAY,LINENAME,STATION,TIME,ENTRIES,EXITS,ENTRY_DIFF,EXIT_DIFF,HR_DELTA,MAX,EN_HRLY_RATE,EX_HRLY_RATE,SUM_PPL
641,1,4567S,GRD CNTRL-42 ST,3854.0625,1.767180e+10,7.405896e+09,139957.0000,122778.3125,1199.4375,863595.0,36152.450521,32535.984375,262735.3125
781,1,BDFMNQRW,34 ST-HERALD SQ,3223.5000,2.368616e+10,2.823926e+10,112486.5000,102142.0000,1075.5000,774360.0,28189.260417,25607.468750,214628.5000
780,1,BDFMNQR,34 ST-HERALD SQ,3265.0000,1.614653e+10,1.717310e+10,109367.7500,99260.7500,1074.3750,773550.0,27802.104167,25498.979167,208628.5000
736,1,ACE,34 ST-PENN STA,3345.1875,2.630517e+09,1.327008e+09,75886.1875,65989.8125,1106.5625,796725.0,19219.812500,16603.963542,141876.0000
746,1,ACENQRS1237W,42 ST-PORT AUTH,2579.7500,3.682319e+10,3.117484e+10,82324.5000,48651.8750,829.5000,597240.0,20771.114583,12168.713542,130976.3750


In [47]:
tues_plotting_data = []
for i, station in enumerate(tues_data['STATION']):
    stop = process.extractOne(station, train_stops_for_mapping, scorer=fuzz.token_set_ratio)
    routes = df_train_stops[df_train_stops['STOP_NAME'] == stop[0]]['DAYTIME_ROUTES']
    line = [li for li in tues_data['LINENAME']]
    line2 = process.extractOne(line[i], routes, scorer=fuzz.token_set_ratio)
    lats = df_train_stops[(df_train_stops['STOP_NAME']==stop[0])&(df_train_stops['DAYTIME_ROUTES']==line2[0])]['LATITUDE']
    longs = df_train_stops[(df_train_stops['STOP_NAME']==stop[0])&(df_train_stops['DAYTIME_ROUTES']==line2[0])]['LONGITUDE']
    tues_plotting_data.append([stop[0], line2[0], [lat for lat in lats][0], [long for  long in longs][0]])

In [48]:
tues_station_coord_df = pd.DataFrame(station_coord, columns=['STATION', 'STOP_NAME', 'LAT', 'LONG'])

In [49]:
tues_station_coord_df.head()

,STATION,STOP_NAME,LAT,LONG
0,34 St - Herald Sq,N Q R W,40.749567,-73.987950
1,1 Av,L,40.730953,-73.981628
2,Beverly Rd,2 5,40.645098,-73.948959
3,Avenue M,B Q,40.617618,-73.959399
4,Grand Central - 42 St,4 5 6,40.751776,-73.976848


In [50]:
tues_geo_data = []
for stations in geo_id:
    station = stations[0][0]
    stop = stations[0][1]
    lat = stations[0][3]
    lon = stations[0][2]
    geo = stations[1]
    tues_geo_data.append([station, stop, lat, lon, geo])
tues_income_df = pd.DataFrame(tues_geo_data, columns=columns)      


In [51]:
tues_income_df.head()

,STATION,STOP_NAME,LATITUDE,LONGITUDE,GEO_ID
0,59 St,N R,40.641362,-74.017881,1500000US360470074002
1,Times Sq - 42 St,1 2 3,40.755290,-73.987495,1500000US360610113001
2,Times Sq - 42 St,N Q R W,40.754672,-73.986754,1500000US360610113001
3,42 St - Bryant Pk,B D F M,40.754222,-73.984569,1500000US360610113001
4,Coney Island - Stillwell Av,D F N Q,40.577422,-73.981233,1500000US360470350002


In [52]:
final_tues_station_selection_df = pd.merge(tues_income_df, income_table, on='GEO_ID', how='left')

In [53]:
final_tues_station_selection_df.head()

,STATION,STOP_NAME,LATITUDE,LONGITUDE,GEO_ID,GEO_ID2,GEO_DISPLAY_LABEL,MEDIAN_INCOME,MARGIN_OF_ERROR
0,59 St,N R,40.641362,-74.017881,1500000US360470074002,360470074002,"Block Group 2, Census Tract 74, Kings County, ...",58804,12572
1,Times Sq - 42 St,1 2 3,40.755290,-73.987495,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
2,Times Sq - 42 St,N Q R W,40.754672,-73.986754,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
3,42 St - Bryant Pk,B D F M,40.754222,-73.984569,1500000US360610113001,360610113001,"Block Group 1, Census Tract 113, New York Coun...",82361,29445
4,Coney Island - Stillwell Av,D F N Q,40.577422,-73.981233,1500000US360470350002,360470350002,"Block Group 2, Census Tract 350, Kings County,...",28359,20779


In [54]:
tues_top_stations = top_stations(number_of_stations, final_tues_station_selection_df)

In [55]:
tues_top_stations

,STATION,STOP_NAME,LATITUDE,LONGITUDE,MEDIAN_INCOME
17,Delancey St,F,40.718611,-73.988114,93643
38,103 St,B C,40.796092,-73.961454,92121
21,Bedford Av,L,40.717304,-73.956872,90370
15,2 Av,F,40.723402,-73.989938,89398
42,Grand St,B D,40.718267,-73.993753,86957
